In [1]:
import networkx as nx
import numpy as np
import time
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [2]:
# --- 1. Graph Loading and Ground Truth Setup (Karate Club) ---
G = nx.karate_club_graph()
num_clusters = 2

# Extract ground truth labels
label_map = {'Mr. Hi': 0, 'Officer': 1}
true_labels = np.array([
    label_map[G.nodes[i]['club']] 
    for i in G.nodes
])

(i) What the Code Does: 
The `spectral_cluster_and_evaluate` function performs Spectral Clustering on a given graph and evaluates the performance of the clustering result. It takes the Laplacian matrix of a graph (`L_matrix`), the true group labels (`true_labels`), and the number of clusters (`num_clusters`) as input. It then executes the Spectral Clustering algorithm, measures the execution runtime, and calculates the Adjusted Rand Index (ARI) to compare the resulting clusters against the true labels. 

(ii) In short, this function serves as a way to apply the spectral clustering algorithm, record the time it takes to perform the clustering 
process (runtime), and to quantify the quality of the clustering result using the Adjusted Rand Index (ARI). 

(iii) How it works:
1) Timing Start: It records the start_time before the computation begins.

2) Eigen-decomposition: It computes the $(k+1)$ smallest eigenvalues and their corresponding eigenvectors of the Laplacian matrix (where $k$ is the `num_clusters`). It uses `np.linalg.eigh` for symmetric matrices, with a fallback try/except block to ensure robust execution. The eigenvectors are then sorted by the magnitude of their eigenvalues.

3) Embedding Creation (Dimensionality Reduction): It discards the first eigenvector (which is trivial and associated with the eigenvalue of $0$) and takes the next $k$ eigenvectors. These $k$ eigenvectors form the new feature space, or embedding, for the graph nodes.

4) K-Means Clustering: It applies the standard K-Means algorithm to the $k$-dimensional embedding. The results of the K-Means clustering are the final `predicted_labels` for the graph nodes.

5) Timing End & Runtime Calculation: It calculates the runtime by subtracting the `start_time` from the current time (`time.time()`).

6) Performance Evaluation: It calculates the `ari_score` by comparing the `predicted_labels` with the `true_labels` using `adjusted_rand_score`.

7) Return: Finally, it returns the calculated `runtime`, `ari_score`, and the `predicted_labels`.

In [3]:
# --- 2. Spectral Clustering Core Function ---
def spectral_cluster_and_evaluate(L_matrix, true_labels, num_clusters):
    """
    Performs spectral clustering, times the process, and calculates the ARI.
    
    Args:
        L_matrix (np.array): The Laplacian matrix (L or L_sym).
        true_labels (np.array): The known ground truth labels.
        num_clusters (int): The 'k' for k-means.
        
    Returns:
        tuple: (runtime_seconds, adjusted_rand_index)
    """
    
    start_time = time.time()
    
    # Eigen-decomposition
    # Use eigsh for symmetric matrices. Request k+1 and discard the first (trivial)
    # Note: We convert to a sparse matrix (csr_matrix) if it's large, but keep it 
    # as a dense array here since the Karate graph is very small.
    try:
        eigenvalues, eigenvectors = eigsh(L_matrix, k=num_clusters + 1, which='SA', sigma=0)
    except Exception as e:
        print(f"Eigsh failed, using numpy.linalg.eigh: {e}")
        eigenvalues, eigenvectors = np.linalg.eigh(L_matrix)
        # Sort by eigenvalue magnitude
        idx = eigenvalues.argsort()
        eigenvectors = eigenvectors[:, idx]
        
    # Discard the first eigenvector (associated with eigenvalue near 0)
    # The first k vectors (Fiedler vector and subsequent) form the embedding.
    embedding = eigenvectors[:, 1:num_clusters+1] 
    
    # k-means clustering on the embedding
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
    predicted_labels = kmeans.fit_predict(embedding)
    
    runtime = time.time() - start_time
    
    # Calculate ARI
    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    
    return runtime, ari_score, predicted_labels

In [4]:
# --- 3. Construct Laplacians ---

# Unnormalized Laplacian (L)
L_unnorm = nx.laplacian_matrix(G).todense()
L_unnorm_array = np.array(L_unnorm)

# Symmetrically Normalized Laplacian (L_sym)
L_sym_norm = nx.normalized_laplacian_matrix(G).todense()
L_sym_norm_array = np.array(L_sym_norm)

In [5]:
# --- 4. Run Comparisons ---
print("--- Running Spectral Clustering Comparison (Karate Club) ---")

# Run 1: Unnormalized Laplacian (L)
runtime_L, ari_L, labels_L = spectral_cluster_and_evaluate(
    L_unnorm_array, true_labels, num_clusters
)

print(f"\n**Unnormalized Laplacian (L):**")
print(f"  > ARI (Clustering Quality): {ari_L:.4f}")
print(f"  > Runtime (Spectral + KMeans): {runtime_L:.4f} seconds")
# print(f"  > Predicted Labels: {labels_L}") # Uncomment to see the labels

# Run 2: Symmetrically Normalized Laplacian (L_sym)
runtime_L_sym, ari_L_sym, labels_L_sym = spectral_cluster_and_evaluate(
    L_sym_norm_array, true_labels, num_clusters
)

print(f"\n**Symmetrically Normalized Laplacian (L_sym):**")
print(f"  > ARI (Clustering Quality): {ari_L_sym:.4f}")
print(f"  > Runtime (Spectral + KMeans): {runtime_L_sym:.4f} seconds")
# print(f"  > Predicted Labels: {labels_L_sym}") # Uncomment to see the labels

print("\n--- End of Single Run ---")

--- Running Spectral Clustering Comparison (Karate Club) ---

**Unnormalized Laplacian (L):**
  > ARI (Clustering Quality): 0.0000
  > Runtime (Spectral + KMeans): 0.0525 seconds

**Symmetrically Normalized Laplacian (L_sym):**
  > ARI (Clustering Quality): 0.0216
  > Runtime (Spectral + KMeans): 0.0142 seconds

--- End of Single Run ---
